#Introducción:
En este cuaderno vamos a intentar analizar la existencia del llamado "ascensor social" en España a partir de los microdatos de la Encuesta de Condiciones de Vida (ECV) del año 2019. Dicho análisis nos sirve como excusa para tratar de mostrar en qué consiste una regresión logística y cómo llevarla a cabo en R.

En primer lugar, debemos definir qué es eso del "ascensor social" y cómo vamos a tratar de analizarlo nosostros. Éste se define como la posibilidad de ascender o descender de clase social. Aunque podemos considerar que la pertenencia a una determinada clase social - si es que éstas existen de forma estanca y perfectamente distinguible - se explica por una combinación de aspectos: nivel económico, cultural, de estudios, etc, nosotros nos vamos a centrar simplemente en el nivel económico.

Para ello, usaremos datos transversales de la ECV del año 2019. Este es el último año - hasta la fecha (2021) - que la ECV ha contado con el modulo temático *Transmisión intergeneracional de la pobreza* - batería de preguntas que nos permiten realizar el análisis deseado.

Los microdatos de la ECV 2019 pueden descargarse en el siguiente link: https://www.ine.es/dyngs/INEbase/es/operacion.htm?c=Estadistica_C&cid=1254736176807&menu=resultados&idp=1254735976608#!tabs-1254736195153. Dichos microdatos están formados por cuatro archivos:
* Fichero D: datos básicos del hogar (*esudb19d.csv*).
* Fichero R: datos básicos de la persona (*esudb19r.csv*).
* Fichero H: datos detallados del hogar (*esudb19h.csv*).
* Fichero P: datos detallados de los adultos (*esudb19p.csv*). En dicho fichero se encuentran las preguntas del módulo *Transmisión intergeneracional de la pobreza*.

Para poder unir los ficheros contamos con una variable de identificación por fichero. Las variables de identificación de los ficheros de hogares (DB030 y HB030) son idénticas; lo mismo para las variables de identificación de los ficheros de personas (RB030 y PB030). Éstas últimas se componen del identificador del hogar y el nº de orden, a dos dígitos, de la persona dentro del hogar.

#¿Cómo determinamos si una persona ha mejorado o no su situación económica?
Para responder a esta pregunta vamos a utilizar las respuestas de los encuestados a dos cuestiones concretas:
1. **Situación económica del hogar cuando el adulto era	adolescente**. Las opciones de respuesta son:
  1. Muy mala
  2. Mala
  3. Moderadamente mala
  4. Moderadamente buena
  5. Buena
  6. Muy buena

Esta pregunta pertenece al módulo *Trasmisión intergeneracional de la pobreza* y, por tanto, se encuentra en el fichero P. Es la pregunta PT190.

2. **Capacidad del hogar para llegar a fin de mes.** Cuyas opciones de respuesta son:
  1. Con mucha dificultad.
  2. Con dificultad.
  3. Con cierta dificultad.
  4. Con cierta facilidad.
  5. Con facilidad.
  6. Con mucha facilidad.

Esta pregunta se refiere al hogar y se encuentra, específicamente, en el fichero H. Es la HS120.

Como vemos, las preguntas no son idénticas, ni siquiera sus opciones de respuesta lo son. Pero sí resultan muy similares y podemos utizarlas como proxy para determinar si una persona ha mejorado o no su situación económica.

Vamos a superponer, pues, lo siguiente: una persona ha mejorado su situación económica, es decir, ha logrado subir en el ascensor social, si ha mejorado en el número de su respuesta de la pregunta 1 a la 2. Por ejemplo: una persona que responde *Muy mala* (opción 1) en la pregunta 1, y *Con cierta dificultad* (opción 3) en la segunda pregunta, aunque su situación econónomica siga sin calificarla de "buena", **sí** que ha mejorado. Por el contrario si el número de la respuesta es el mismo o menor en la pregunta 2 que en la 1, dicha persona **no** habrá mejorado.

De esta manera, clasificaremos a los encuestados en dos categorías: 1) Personas que han mejorado - *han subido con el ascensor social*; 2) Personas que no han mejorado - *no han subido con el ascensor social*. Dicha sencilla categorización puede resultar simplista, pero nos permitirá llevar a cabo una regresión logística binaria, la más sencilla y, por ello, la más fácil de entender y realizar.

Esta variable binaria será la variable dependiente de nuestra regresión logística.

Para subir un archivo de datos (por ejemplo, un .csv) a Google Colab y trabajar con R debemos hacer lo siguiente:

1. Clicamos en el icono de la derecha de la pantalla que tiene forma de carpeta y se abre un desplegable a su derecha.
2. Clicamos en el icono que nos muestra una carpeta con una flecha hacia arriba en su interior.
3. Se abrirá una carpeta para que busquemos el archivo. Una vez seleccionado, le damos a "Abrir".
4. Esperamos a que se cargue y, una vez cargado, ya podemos usar la función de R apropiada para leer dicho archivo.

(Esta operación hay de carga hay que realizarla cada vez que se abra el cuaderno)

In [8]:
datos_Fichero_P <- read.csv("esudb19p.csv", sep=",")
head(datos_Fichero_P)

,PB010,PB020,PB030,PB040,PB040_F,PB100,PB100_F,PB110,PB110_F,PB120,⋯,PT180,PT180_F,PT190,PT190_F,PT260,PT260_F,PT270,PT270_F,PT280,PT280_F
,<int>,<chr>,<int>,<dbl>,<int>,<int>,<int>,<int>,<int>,<int>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
1,2019,ES,101,906.1583,1,12,1,2019,1,6,⋯,NA,-6,NA,-6,NA,-6,NA,-6,NA,-6
2,2019,ES,102,906.1583,1,12,1,2019,1,3,⋯,NA,-6,NA,-6,NA,-6,NA,-6,NA,-6
3,2019,ES,201,1227.6887,1,12,1,2019,1,5,⋯,NA,-6,NA,-6,NA,-6,NA,-6,NA,-6
4,2019,ES,202,1233.6804,1,12,1,2019,1,6,⋯,NA,-1,NA,-1,NA,-1,NA,-1,NA,-1
5,2019,ES,301,2559.3599,1,12,1,2019,1,8,⋯,5,1,4,1,1,1,1,1,3,1
6,2019,ES,302,2600.2733,1,12,1,2019,1,9,⋯,5,1,3,1,1,1,1,1,2,1


In [2]:
cat("El número de filas del dataframe datos_Fichero_P es", nrow(datos_Fichero_P), "y el de columnas es", ncol(datos_Fichero_P))

El número de filas del dataframe datos_Fichero_P es 33376 y el de columnas es 244

In [25]:
datos_Fichero_H <- read.csv("esudb19h.csv", sep=",")
head(datos_Fichero_H)

,HB010,HB020,HB030,HB050,HB050_F,HB060,HB060_F,HB070,HB070_F,HB080,⋯,HD080,HD080_F,HI010T,HI010T_F,HI020T,HI020T_F,HI030T,HI030T_F,HI040T,HI040T_F
,<int>,<chr>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
1,2019,ES,1,12,1,2019,1,101,1,101,⋯,1,1,2,1,NA,-2,NA,-2,2,1
2,2019,ES,2,12,1,2019,1,201,1,201,⋯,1,1,2,1,NA,-2,NA,-2,2,1
3,2019,ES,3,12,1,2019,1,301,1,301,⋯,2,1,3,1,NA,-2,5,1,NA,-1
4,2019,ES,4,12,1,2019,1,401,1,401,⋯,2,1,3,1,NA,-2,9,1,3,1
5,2019,ES,6,11,1,2019,1,601,1,602,⋯,2,1,3,1,NA,-2,1,1,2,1
6,2019,ES,7,11,1,2019,1,701,1,701,⋯,1,1,2,1,NA,-2,NA,-2,2,1


In [26]:
cat("El número de filas del dataframe datos_Fichero_H es", nrow(datos_Fichero_H), "y el de columnas es", ncol(datos_Fichero_H))

El número de filas del dataframe datos_Fichero_H es 15887 y el de columnas es 161

Vamos a crear una variable en el conjunto de datos *datos_Fichero_P* que sea igual a la variable identificadora del hogar HB030. Para ello, debemos quitar los dos últimos dígitos a la variable PB030. De esta forma, tendremos dos variables identificadoras idénticas que nos permitirán realizar la unión entre tablas.

In [27]:
#Para quitar los dos últimos dígitos dividimos entre 100, quedándonos solo con la parte entera:
datos_Fichero_P$HB030 = datos_Fichero_P$PB030 %/% 100

Llevamos a cabo, entonces, la unión de *dataframes*, usando las variables HB030 de cada *dataframe* como nexo de unión:

In [34]:
datos_conjuntos <- merge(x=datos_Fichero_P, y=datos_Fichero_H, by="HB030")
head(datos_conjuntos)

,HB030,PB010,PB020,PB030,PB040,PB040_F,PB100,PB100_F,PB110,PB110_F,⋯,HD080,HD080_F,HI010T,HI010T_F,HI020T,HI020T_F,HI030T,HI030T_F,HI040T,HI040T_F
,<dbl>,<int>,<chr>,<int>,<dbl>,<int>,<int>,<int>,<int>,<int>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
1,1,2019,ES,101,906.1583,1,12,1,2019,1,⋯,1,1,2,1,NA,-2,NA,-2,2,1
2,1,2019,ES,102,906.1583,1,12,1,2019,1,⋯,1,1,2,1,NA,-2,NA,-2,2,1
3,2,2019,ES,201,1227.6887,1,12,1,2019,1,⋯,1,1,2,1,NA,-2,NA,-2,2,1
4,2,2019,ES,202,1233.6804,1,12,1,2019,1,⋯,1,1,2,1,NA,-2,NA,-2,2,1
5,3,2019,ES,301,2559.3599,1,12,1,2019,1,⋯,2,1,3,1,NA,-2,5,1,NA,-1
6,3,2019,ES,302,2600.2733,1,12,1,2019,1,⋯,2,1,3,1,NA,-2,5,1,NA,-1


In [29]:
cat("El número de filas del dataframe datos_conjuntos es", nrow(datos_conjuntos), "y el de columnas es", ncol(datos_conjuntos))

El número de filas del dataframe datos_conjuntos es 33376 y el de columnas es 405

Como vemos, tenemos un nuevo *dataframe* cuyo número de filas es igual al número de filas del fichero P (el más grande, ya que hay más personas que hogares), y cuyo número de columnas es igual a la suma de las variables de los ficheros P y H.

A continuación, creamos la variable MSE (*Mejora_Situación_Económica*). Dicha variable tendrá valor 1 si la situación económica de cada persona ha mejorado con los años y valor 0, si no lo ha hecho:

In [32]:
datos_conjuntos$MSE <- ifelse(datos_conjuntos$PT190 < datos_conjuntos$HS120, 1, 0)
#cat("El número de personas que han mejorado su situación económica es de", nrow(datos_conjuntos[datos_conjuntos$MSE == 1,]), ". Mientras que el resto,", nrow(datos_conjuntos[datos_conjuntos$MSE == 0,]), ", no han mejorado su situación económica.")

In [35]:
summary(datos_Fichero_P$PT190)
summary(datos_Fichero_H$HS120)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max.    NA's 
  1.000   3.000   4.000   3.982   5.000   6.000   15900 

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max.    NA's 
  1.000   3.000   4.000   3.446   4.000   6.000      15 